In [1]:
# !pip install pandas
# !pip install seaborn
# !pip install sklearn
# !pip install pyenchant
# !apt-get install libenchant1c2a

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing import sequence
import os 
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
from tensorflow.keras.layers import Input,Bidirectional,GlobalAveragePooling1D,GlobalMaxPooling1D,concatenate,LeakyReLU
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

os.chdir('/content/drive/MyDrive/NeuralResearch/wine-net/')

# os.chdir('/content/drive/MyDrive/MSCA_31009/Final_Project/')

def remove_punctuation(s):
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

  # Removing punctuations in string
  # Using loop + punctuation string
  for ele in s:
      if ele in punc:
          s = s.replace(ele, "")
  return s

def clean_description(review, swap):
  terms = review.split()
  for x in terms:
    try: x = swap[('t_' + str(x))]
    except KeyError: x = ''
  return ' '.join(terms)


In [4]:
wines = pd.read_csv('winemag-data-130k-v2.csv')
wines = wines.drop(columns = ['Unnamed: 0'])

wines2 = pd.read_csv('winemag-data_first150k.csv')
wines2.head()

# Intersect and append two review tables
columns = [value for value in wines.columns if value in wines2.columns] # intersection(wines.columns, wines2.columns)
wines = wines[columns]
wines2 = wines2[columns]
wines = pd.concat([wines, wines2]).drop(columns = ['designation', 'winery'])
wines['d_length'] = wines.description.apply(lambda x: len(str(x))).astype(int)

wines = wines[((wines.d_length > 75) & (wines.d_length < 450))]

wines = wines.sample(n=20000)

# Imputation steps
median_price = wines.price.median()
median_points = wines.points.median()

# Impute variables
wines.price = wines.price.fillna(median_price)
wines.points = wines.points.fillna(median_points)
wines = wines.fillna('')
wines.price = wines.price.astype(int)
wines.description = wines.description.apply(lambda x: remove_punctuation(x.lower()))

Y = wines.variety
output_shape = Y.nunique()
Y0_price = wines.price
Y1_points = wines.points.apply(lambda x: 1 if x > 90 else 0)

Y = [0 if wines['variety'].iloc[x] in ['Pinot Noir','Cabernet Sauvignon','Red Blend','Bordeaux-style Red Blend','Syrah','Merlot'] else 1 for x in range(len(wines['variety']))]
Y = pd.Series(Y)

wines = wines.drop(columns=['variety', 'price', 'points', 'region_1', 'region_2', 'd_length'])
wines.head()

,country,description,province
117472,Chile,this wine is the perfect model for midlevel ch...,Colchagua Valley
88967,Portugal,round and ripe this balances rich white and ye...,Setubal
87490,US,a dusty drink of blackberry lean and structure...,California
51997,US,another in the new nn line the sullenger featu...,California
43655,Germany,dusty earth and mineral tones lend contrast to...,Rheingau


In [5]:
text = wines.description
tfidf = TfidfVectorizer() 
review_vector = tfidf.fit_transform(text)
words = tfidf.get_feature_names_out()
rev_array = review_vector.toarray()
words_df = pd.DataFrame(rev_array, columns = words)

num_words = len(words)

import enchant
enchant.list_languages()
eng = enchant.Dict("en_US")
drops = []
for column in words_df.columns:
    english = eng.check(column)
    if any(map(str.isdigit, column)): 
        drops.append(column)
    if not(english):
        drops.append(column)
words_df = words_df.drop(columns = drops)
reduced_words = len(words_df.columns)
print('Reduced words by ', str(round(1-(reduced_words/num_words),2)*100), '%')
print('Reduced word count: ', str(reduced_words))
words_df.head()

Reduced words by  41.0 %
Reduced word count:  9283


,abbey,ability,able,ably,abnormal,abnormally,abound,abounds,about,above,...,zesty,zinfandel,zing,zinging,zingy,zip,zippy,zips,zone,zucchini
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from keras.preprocessing.text import Tokenizer
X = wines.description.to_numpy()
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X)
num_words = len(tokenizer.word_index)+1
X = tokenizer.texts_to_sequences(X)
max_log_length = 1024
X_processed = sequence.pad_sequences(X, maxlen=max_log_length)
X_processed

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(Y)
Y_processed = encoder.transform(Y)
Y_processed = Y.to_numpy()

## Variety Classifier

In [7]:
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict

X_train, X_test, y_train, y_test = train_test_split(X_processed, Y_processed, test_size=0.25, random_state=0)

Y.value_counts()
input_dim = X_train.shape[1]

## RNN Build I

In [8]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=num_words, output_dim=1024,input_length=max_log_length))
model.add(tf.keras.layers.LSTM(units=32, recurrent_dropout=0.5))
model.add(tf.keras.layers.Dense(units=64,activation='relu'))
model.add(tf.keras.layers.Dense(units=32,activation='relu'))
model.add(tf.keras.layers.Dense(units=1,activation='relu'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1024, 1024)        16788480  
                                                                 
 lstm (LSTM)                 (None, 32)                135296    
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 16,928,001
Trainable params: 16,928,001
Non-trainable params: 0
_________________________________________________________________


## RNN Build II

In [9]:
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Embedding(input_dim=num_words, output_dim=1024,input_length=max_log_length))
model2.add(tf.keras.layers.LSTM(units=32, recurrent_dropout=0.5))
model2.add(tf.keras.layers.Dense(units=1,activation='relu'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1024, 1024)        16788480  
                                                                 
 lstm_1 (LSTM)               (None, 32)                135296    
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 16,923,809
Trainable params: 16,923,809
Non-trainable params: 0
_________________________________________________________________


## Fit & Evaluation


In [10]:
model.fit(x=X_train, y = y_train, batch_size = 256, epochs = 4)

Epoch 1/3
39/39 [==============================] - 294s 7s/step - loss: 0.9896 - accuracy: 0.5354
Epoch 2/3
39/39 [==============================] - 279s 7s/step - loss: 0.5195 - accuracy: 0.7324
Epoch 3/3
39/39 [==============================] - 275s 7s/step - loss: 0.3722 - accuracy: 0.8364


In [11]:
model.evaluate(X_test, y_test, batch_size=256)

13/13 [==============================] - 12s 852ms/step - loss: 0.5385 - accuracy: 0.7643


[0.5385441184043884, 0.7643076777458191]

In [ ]:
model2.fit(x=X_train, y = y_train, batch_size = 256, epochs = 4)

Epoch 1/3
 7/39 [====>.........................] - ETA: 3:29 - loss: 2.1403 - accuracy: 0.3499

In [ ]:
model2.evaluate(X_test, y_test, batch_size=256)